## Exploring the variation of Vertical flux in SHARP regions during solar cycle

In [ ]:
from astropy.io import fits
from astropy.coordinates import SkyCoord
import astropy.units as u
from datetime import datetime as dt_obj
import drms
from IPython.display import Image
import json
import matplotlib.pyplot as plt, matplotlib.ticker as mtick
from matplotlib.dates import *
import matplotlib.image as mpimg
import numpy as np
import pandas as pd
from pandas.plotting import register_matplotlib_converters
import os.path
from scipy import stats
import sunpy.map
from sunpy.coordinates import frames
from sunpy.cm import color_tables as ct
from sunpy.timeseries import TimeSeries
from sunpy.time import TimeRange, parse_time
from sunpy.net import hek, Fido, attrs as a
from tqdm import tqdm, trange
import urllib
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
t1    = '2018.01.01_00:00:00_TAI'
t2    = '2019.08.08_07:00:00_TAI'
dt1   = drms.to_datetime(t1)
dt2   = drms.to_datetime(t2)
delt  = int((dt2 - dt1).total_seconds() // 3600//24)
email ='ap0162@uah.edu'  
ds    ='hmi.sharp_cea_720s'  
dt    = t1
delt1 = delt
delt2 = '1h'
key   = 'HARPNUM, NOAA_AR, T_OBS, T_FRST1, T_FRST, CRLN_OBS,CRLT_OBS, AREA_ACR, LAT_FWT, LON_FWT'
c     = drms.Client(email=email, verbose = True)
keys  = c.query('{0}[][{1}/{2}d@{3}]'.format(ds,dt,delt1,delt2), key=key)
#keys
keys.info()

In [ ]:
#=========== INPUT BLOCK ==================
datadir     = '/home/jovyan/'  # data directory
#ardatafile  = 'hmi_ArData2012_2017.dat'                # HMI SHARP data file
max_lon     = 45                                       # maximum value for abs(longitude) to select only disc center objects
#index       = 12                                       # index of the AR from the selected NOAA list
# harp index range : 0 - 3054
# noaa index range : 0 - 980
#email       ='ap0162@uah.edu'                          # email id for the JSOC client
ds          ='hmi.sharp_cea_720s'                      # HMI dataseries
#ar_kw       = ['T_REC', 'CROTA2','CRPIX1', 'CRPIX2', 
#               'CDELT1', 'CDELT2', 'CRVAL1', 'CRVAL2'] # Keywords for the Magnetic field 
ar_segments = ['Br','Bp','Bt']                         # Segments to be downloaded

In [ ]:
# Reading the SHARP dataset and noting the headers
#ardata = pd.read_csv(datadir + ardatafile,sep='\t')
ardata = keys
#print(ardata.info())
#print(ardata.columns)
#print(ardata.shape)
#ardata.head(2)

# Selecting only those regions that fall with in the +/- 45 degree Longitude
ardata_cen = ardata[abs(ardata['LON_FWT']) < max_lon]
#print(ardata_cen.info())
#print(ardata_cen.shape)

# Saving all the HARP numbers in harp variable
harp=ardata_cen['HARPNUM'].drop_duplicates().reset_index(drop=True)
#print(harp.head(2))
#print(harp.tail(2))

harp_size = harp.size

# Saving all the NOAA numbers in noaa variable
noaa = ardata_cen['NOAA_AR'].drop_duplicates()
noaa = noaa[noaa!=0].reset_index(drop=True)
#print(noaa.head(2))
#print(noaa.tail(2))
noaa_size = noaa.size

# Function to gather all the data for a given HARP number
def harp_data(harp_index):
    harp_data = ardata_cen[ardata_cen['HARPNUM']==harp[harp_index]]
    harp_data = harp_data.reset_index(drop=True)
    return harp_data

# Function to gather all the data for a given NOAA number
def noaa_data(noaa_index):
    noaa_data = ardata_cen[ardata_cen['NOAA_AR']==noaa[noaa_index]]
    noaa_data = noaa_data.reset_index(drop=True)
    return noaa_data

In [ ]:
# Selecting the region based on either the HARP or NOAA number
def sharp_flux(index):
    #harps = noaa_data(index)
    harps = harp_data(index)
    #harps.head(1)

    # Finding the first and last observation time for the selected region
    harpf = harps['HARPNUM'][0]
    noaaf = harps['NOAA_AR'][0]
    t1 = harps['T_OBS'].iloc[0]
    t2 = harps['T_OBS'].iloc[-1]
    dt1 = drms.to_datetime(t1)
    dt2 = drms.to_datetime(t2)
    delt = int((dt2 - dt1).total_seconds() // 3600)

    # Image of HARP regions at the start time
    #print('Start time = {}'.format(t1))
    #print('HARP = {0}, NOAA = {1}'.format(harpf,noaaf))
    date_im1 = dt1.strftime('%Y/%m/%d')
    date_time_im1 = dt1.strftime('%Y.%m.%d_%H:00:00_TAI')
    url_string = 'http://jsoc.stanford.edu/doc/data/hmi/harp/harp_definitive/'
    url1 = url_string + date_im1 + '/harp.'+date_time_im1+'.png'
    #Image(url1)

    # Image of HARP regions at the end time

    #print('HARP = {0}, NOAA = {1}'.format(harpf,noaaf))
    #print('End time   = {}'.format(t2))
    date_im2 = dt2.strftime('%Y/%m/%d')
    date_time_im2 = dt2.strftime('%Y.%m.%d_%H:00:00_TAI')
    url2 = url_string + date_im2 + '/harp.'+date_time_im2+'.png'
    #Image(url2)

    # Plotting the Unsigned Flux between start and end time of the observation
    c = drms.Client(email=email, verbose = True)
    keys, segments = c.query('{0}[{1}][{2}/{3}h]'.format(ds,harpf,dt1,delt),
                             key='T_REC, USFLUX, ERRVF', seg='Br', rec_index=True)
    #keys.info()

    nplot = 1 # plot every nth point <-- change if needed

    def parse_tai_string(tstr,datetime=True):
        year   = int(tstr[:4])
        month  = int(tstr[5:7])
        day    = int(tstr[8:10])
        hour   = int(tstr[11:13])
        minute = int(tstr[14:16])
        if datetime: return dt_obj(year,month,day,hour,minute)
        else: return year,month,day,hour,minute
    t_rec = np.array([parse_tai_string(keys.T_REC[i],datetime=True) for i in range(keys.T_REC.size)])

    fig, ax = plt.subplots(figsize=(12,8))      # define the size of the figure
    orangered = (1.0,0.27,0,1.0)                # create an orange-red color
    # define some style elements
    marker_style = dict(linestyle='--', markersize=4, fillstyle='full',
                        color=orangered,markeredgecolor=orangered)
    text_style = dict(fontsize=16, fontdict={'family': 'monospace'})
    ax.tick_params(labelsize=14)
    ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))
    # ascribe the data to the axes
    ax.plot(t_rec[::nplot], (keys.USFLUX)[::nplot]/(1e22),'o',**marker_style)
    ax.errorbar(t_rec[::nplot], (keys.USFLUX)[::nplot]/(1e22),
                yerr=(keys.ERRVF[::nplot])/(1e22), linestyle='',color=orangered)
    # format the x-axis with universal time
    locator = AutoDateLocator()
    locator.intervald[HOURLY] = [12] # only show every 12 hours
    formatter = DateFormatter('%d_%H')
    #formatter = DateFormatter('%H')
    ax.xaxis.set_major_locator(locator)
    ax.xaxis.set_major_formatter(formatter)
    # set yrange 
    #ax.set_ylim([2.4,2.8])
    # label the axes and the plot
    ax.set_xlabel('date_time in UT',**text_style);
    ax.set_ylabel('maxwells x 1e22',**text_style);
    ax.set_title('total unsigned flux starting at '+str(t_rec[0])+' UT',**text_style);
    # annotate the plot with a start time
    ax.grid(True,axis='y')

    # Find the time when the unsigned flux is maximum
    #max_flux = keys.USFLUX.max()
    #rec_max = keys.USFLUX.idxmax()
    max_flux = keys.USFLUX.sort_values(ascending=False)[0] #<---- incase of outlier change to higher values
    rec_max=keys.USFLUX[keys.USFLUX==max_flux].index[0]
    key_max = keys.loc[rec_max]
    t_max = drms.to_datetime(keys.T_REC[rec_max])
    t_maxf = t_max.strftime('%Y.%m.%d_%H:%M:%S_TAI')
    ax.axvline(t_max)
    fname='{0}_{1}_{2}_{3}'.format(ds,harpf,date_time_im1,date_time_im2)
    #print(plt_name)
    fig.savefig(fname+'.png', dpi=150,transparent=False,
                bbox_inches = 'tight',pad_inches = 0.1)
    plt.close(fig)
    # print('\n')
    # print('------ Summary -------')
    # print('HARP = {}'.format(harpf))
    # print('NOAA = {}'.format(noaaf))
    # print('Start time = {}'.format(t1))
    # print('End time   = {}'.format(t2))
    # print('Duration of passage = {0} h'.format(delt))
    # print('Dataset for Maximum flux:', rec_max)
    # print('Time of Max Flux:', t_max)
    # print('Maximum flux =', max_flux)
    # print('-'*25)
    # print('\n')
    # write out the summary to a txt file
    fname2='{0}_us_flux_2018_2019'.format(ds)
    if not os.path.isfile(datadir + fname2+'.txt'):
        with open(fname2 + '.txt', 'w') as f: 
            f.write('HARP\tNOAA\tStart Time\tEnd Time\tDuration\tTime_max\tMax Flux\n')
            f.write('{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}'.format(harpf,noaaf,t1,t2,delt,t_max,max_flux))
    else:
        with open(fname2 + '.txt', 'a') as f: 
            f.write('\n{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6}'.format(harpf,noaaf,t1,t2,delt,t_max,max_flux))
    

    # Getting the keyword info
    #hmi_k = c.query('%s[%d][%s]' % (ds, harpf, t_maxf), key=ar_kw, rec_index=True)
    ar_kw = c.keys(ds)
    hmi_k = c.query('%s[%d][%s]' % (ds, harpf, t_maxf), key =ar_kw ,rec_index=True)
    hmi_si = c.info(ds)
    #hmi_si.segments
    clon=(hmi_k.CRVAL1[0])
    clat=(hmi_k.CRVAL2[0])
    ctime=hmi_k.T_REC[0]
    nx=(hmi_k.CRPIX1[0]*2)
    ny=(hmi_k.CRPIX2[0]*2)
    #cor = SkyCoord(clon*u.deg, clat*u.deg, frame=frames.HeliographicCarrington,obstime=ctime)
    #hpc=cor.transform_to(frames.Helioprojective)

    # Requesting the export of Br, Bp, Bt at the time of maximum flux
    req = hmi_k.CRPIX1.idxmin()

    #print(req)

    #print(req)
    exp_query = '%s{%s}' % (req, ','.join(ar_segments) )
    print(exp_query)

    r = c.export(exp_query,protocol='fits')

    # Setting the name convention for saving the fits file
    t_rec = drms.to_datetime(hmi_k.T_REC[0])
    t_rec_str = t_rec.strftime('%Y%m%d_%H%M%S_TAI')
    fname_mask = '{series}.{sharpnum}.{tstr}.{segment}.fits'
    fnames = {
        s: fname_mask.format(
            series=ds, sharpnum=harpf, tstr=t_rec_str, segment=s)
        for s in ar_segments}
    #print(fnames['Br'])

    # Check for files and download if absent
    if not os.path.isfile(datadir + fnames['Br']):
        dl = r.download(datadir)

    print('-'*20)
    print('T_REC = ', ctime)
    print('Clon = {0:5.2f}'.format(clon))
    print('Clat = {0:5.2f}'.format(clat))
    print('nx = ', int(nx))
    print('ny = ', int(ny))
    print('-'*20)

    # Define function to read fits data
    def read_fits_data(fname):
        """Reads FITS data and fixes/ignores any non-standard FITS keywords."""
        hdulist = fits.open(fname)
        hdulist.verify('silentfix+warn')
        return hdulist[1].data

    # Reading the fits files
    br = read_fits_data(datadir+fnames['Br'])
    bt = read_fits_data(datadir+fnames['Bt'])
    bp = read_fits_data(datadir+fnames['Bp'])

    # Converting to Cartesian values
    bx = bp
    by = -bt
    bz = br
    #print(bz.shape)

    # Plot Bz 
    fig = plt.figure(figsize=(8,8))
    plt.imshow(bz,cmap='gray',origin='lower')
    plt.title(bz.shape)
    #plt.show()
    suff=fnames['Br'][:-5]
    plt.savefig(suff+'.png', dpi=150,transparent=False,
                bbox_inches = 'tight',pad_inches = 0.1)
    plt.close(fig)
    return None

In [ ]:
sharp_flux(1)

In [ ]:
#skip_values = [144,152,154,155,156,158,159,160,421,428,431]
skip_values = []
for index in range(harp_size):
    print('index = ',index)
    try:
        sharp_flux(index)
    except:
        skip_values.append(index)
        continue
    

In [ ]:
print(skip_values)

In [ ]:
# Image of the SHARP region during the maximum flux

# protocol_args = {'ct': 'HMI_mag.lut', 'min': -3000, 'max': 3000}
# r = c.export('{0}[{1}][{2}]'.format(ds,harpf,t_maxf)+'{Br}', protocol='jpg',protocol_args=protocol_args,email=email)
# print(r.request_url)
# image_url = r.urls['url'][0]
# print('The image is at this url: ',image_url,'.')
# Image(url=image_url)

# Generating a movie of the SHARP region during the full passage

# r = c.export('{0}[{1}][{2}/{3}h]'.format(ds,harpf,date_time_im1,delt)+'{Br}',protocol='mpg', protocol_args=protocol_args, email=email)
# print(r.request_url)
# print("You can download the movie from here:", r.urls['url'][0])

### Downloading the HMI-SHARP-CEA fits files for Br, Bt and Bp

In [ ]:
# # Downloading the HMI-SHARP-CEA fits files for Br, Bt and Bp

# # Getting the keyword info
# hmi_k = c.query('%s[%d][%s]' % (ds, harpf, t_maxf), key=ar_kw, rec_index=True)
# hmi_si = c.info(ds)
# #hmi_si.segments
# clon=(hmi_k.CRVAL1[0])
# clat=(hmi_k.CRVAL2[0])
# ctime=hmi_k.T_REC[0]
# nx=(hmi_k.CRPIX1[0]*2)
# ny=(hmi_k.CRPIX2[0]*2)
# #cor = SkyCoord(clon*u.deg, clat*u.deg, frame=frames.HeliographicCarrington,obstime=ctime)
# #hpc=cor.transform_to(frames.Helioprojective)

# # Requesting the export of Br, Bp, Bt at the time of maximum flux
# req = hmi_k.CRPIX1.idxmin()
# #print(req)
# exp_query = '%s{%s}' % (req, ','.join(ar_segments))
# #print(exp_query)
# r = c.export(exp_query)

# # Setting the name convention for saving the fits file
# t_rec = drms.to_datetime(hmi_k.T_REC[0])
# t_rec_str = t_rec.strftime('%Y%m%d_%H%M%S_TAI')
# fname_mask = '{series}.{sharpnum}.{tstr}.{segment}.fits'
# fnames = {
#     s: fname_mask.format(
#         series=ds, sharpnum=harpf, tstr=t_rec_str, segment=s)
#     for s in ar_segments}
# #print(fnames['Br'])

# # Check for files and download if absent
# if not os.path.isfile(datadir + fnames['Br']):
#     dl = r.download(datadir)

# print('-'*20)
# print('T_REC = ', ctime)
# print('Clon = {0:5.2f}'.format(clon))
# print('Clat = {0:5.2f}'.format(clat))
# print('nx = ', int(nx))
# print('ny = ', int(ny))
# print('-'*20)

# # Define function to read fits data
# def read_fits_data(fname):
#     """Reads FITS data and fixes/ignores any non-standard FITS keywords."""
#     hdulist = fits.open(fname)
#     hdulist.verify('silentfix+warn')
#     return hdulist[1].data

# # Reading the fits files
# br = read_fits_data(datadir+fnames['Br'])
# bt = read_fits_data(datadir+fnames['Bt'])
# bp = read_fits_data(datadir+fnames['Bp'])

# # Converting to Cartesian values
# bx = bp
# by = -bt
# bz = br
# #print(bz.shape)

# # Plot Bz 
# fig = plt.figure(figsize=(8,8))
# plt.imshow(bz,cmap='gray',origin='lower')
# plt.title(bz.shape)
# #plt.show()
# suff=fnames['Br'][:-5]
# plt.savefig(suff+'.png', dpi=150,transparent=False,
#             bbox_inches = 'tight',pad_inches = 0.1)